<a href="https://colab.research.google.com/github/m-soro/Search-Storeroom/blob/main/Search_Storeroom_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ### Search Storeroom
try:
    import pandas as pd
    import re
    import xlsxwriter
    from tika import parser
    from google.colab import files
    import datetime
except:
    !pip install tika
    !pip install xlsxwriter

fn = '99 Food Storeroom.pdf' 

raw = parser.from_file(fn)
content = raw['content'].replace('\n','').split('_________')[2:]

# item_num = re.findall(r'\d+', two)[-1]
# item_num

# desc = re.findall(r'\D+', two)[2][:-2]
# desc

# price = re.findall(r'\$\d+(?:\.\d+)?', two)[0]
# price

# case_break = re.findall(r'\(([^\)]+)\)',two)[0]
# case_break

# unit = re.findall(r'\D+', str(two).replace(to_remove,'').replace('Location',''))[-1])
# unit

item_num = []
for line in content:
    remove = re.findall(r'\(([^\)]+)\)',line)[0]
    line = line.replace(remove,'')
    item_num.append(re.findall(r'\d+', line)[-1])

case_breaks=[]
for line in content:
    case_breaks.append(re.findall(r'\(([^\)]+)\)',line)[0])

price = []
for line in content:
    price.append(re.findall(r'\$\d+(?:\.\d+)?', line)[0])

desc = []
for line in content:
    desc.append(re.findall(r'\D+', line)[2][:-2])

unit = []
to_remove = 'OutletUnitCountCostDescriptionInventoryInv.ProductProduct'
for line in content:
    unit.append((re.findall(r'\D+', str(line).replace(to_remove,'').replace('Location',''))[-1]))

df = pd.DataFrame(item_num)
df['Desc'] = desc
df['Unit'] = unit
df['Case_Break'] = case_breaks
df['Price'] = price   

df.columns = ['Item_Num', 'Desc', 'Unit','Case_Break','Price']

In [ ]:
#@title Search by Item Description { run: "auto", vertical-output: true }
input = "bread" #@param {type:"string"}
r = df[df['Desc'].str.contains((input).title())]

import ipywidgets as widgets

data = r

checkboxes = [widgets.Checkbox(layout=widgets.Layout(width='150%'),value=False, description=str(label[1][0] + '-'+ str(label).replace('Desc          ','').split('\n')[1])) for label in r.iterrows()]

output = widgets.VBox(children=checkboxes)

display(output)




In [ ]:
#@title Clear the Cart
selected_data = []


In [ ]:
#@title Add to Cart { run: "auto" }
for i in range(0, len(checkboxes)):
    if checkboxes[i].value == True:
        # selected_data = selected_data + [checkboxes[i].description]
        selected_data.append(checkboxes[i].description.split('-')[0])

In [ ]:
#@title Build/Save/Download the Template
template_name = "hungry jeff" #@param {type:"string"}
template = pd.DataFrame(selected_data)

desc = []
for n in template[0]:
    desc.append(df.query('Item_Num == @n').Desc.values[0])

template['Desc'] = desc


unit = []
for n in template[0]:
    unit.append(df.query('Item_Num == @n').Unit.values[0])

template['Unit'] = unit

# Doesn't need case breaks
# case_break = []
# for n in template[0]:
#     case_break.append(df.query('Item_Num == @n').Case_Break.values[0])

# template['Case_Break'] = case_break

template['Order Qty'] = ' '

template.columns = ['Item_Num', 'Description','Unit', 'Order Qty'] 

sheet_name = template_name
d = datetime.datetime.now().strftime("%m.%d.%y")
writer = pd.ExcelWriter(sheet_name + d +'.xlsx',engine='xlsxwriter')
template.to_excel(writer,sheet_name=sheet_name,index=False)
worksheet = writer.sheets[sheet_name]
worksheet.set_column('B:B', template['Description'].str.len().max())
writer.save()
files.download(sheet_name + d +'.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Search by Item Number { run: "auto", vertical-output: true }
input = "" #@param {type:"string"}
df[df['Item_Num'].str.contains(input)] 